In [55]:
import os
import sys
import pandas as pd
import pandasql as pdsql


from bkcharts import Donut
from bokeh.io import output_notebook, show

# Do this trick to add the new Beamly functionality
pythonModsRoot = '/Users/carlos.aguilar/Google Drive/PythonDev/Coding/BeamlyPython'
if pythonModsRoot not in sys.path:
    sys.path.append(pythonModsRoot)

import redshiftSqlAlchemy as rsa
import carlosUtils        as cu
import bokehUtils         as bk

from IPython.display import Markdown, display, HTML
def printmd(string):
    display(Markdown(string))


In [56]:
numCampaigns = 20;
cutOffPerc   = 96.0;


In [57]:
# set the table and schema
schemaName = 'adform'
tableName  = 'impressionsExtended'

# set the name of the variables to break-down
breakDownKeyWords = ['campaign_id', 'yyyy_mm_dd', \
'publisher_domain', 'bannerid', 'bannerType', \
'client_id', 'device_type_id', 'placement_id_activity_id', 'tag_id', \
'city_id' ];

# flags to set the outputs
saveFigure    = True;
saveDFToExcel = True;

In [58]:
# connect to the Redshift box
user     = 'carlos_aguilar'
password = 'MdogDI64j6vH90g973'
dbname   = 'adform'
host     = 'adform-ops.c7dxcjhlundm.eu-central-1.redshift.amazonaws.com'

rs = rsa.RedshiftAlchemy(user=user, password=password, 
    database=dbname, host=host)

In [59]:
sqlQuery = '''SELECT count(*) as numRecords 
    from {}.{}'''.format(schemaName, tableName)
df = rs.query2DF(sqlQuery);
printmd('Analysing **' + schemaName + '.' + tableName + '** (' +  str(df.numrecords[0]) + ' records)' )

Analysing **adform.impressionsExtended** (195387010 records)

In [60]:
# This snippet breaks down the variables and writes the counts to a file
printmd('_Break down variables_ from ' +  schemaName +'.' + tableName)
for currentKey in breakDownKeyWords:
    sqlQuery = '''SELECT count(distinct({})) as numRecords 
        from {}.{}'''.format(currentKey, schemaName, tableName)
    df = rs.query2DF(sqlQuery);
    currentText = '**' + currentKey + '**' + ' has got **' + str(df.numrecords[0]) + '** unique values...'
    printmd(currentText)

_Break down variables_ from adform.impressionsExtended

**campaign_id** has got **247** unique values...

**yyyy_mm_dd** has got **66** unique values...

**publisher_domain** has got **276148** unique values...

**bannerid** has got **1454** unique values...

...

**client_id** has got **37** unique values...

**device_type_id** has got **5** unique values...

**placement_id_activity_id** has got **1636** unique values...

**tag_id** has got **3091** unique values...

**city_id** has got **220894** unique values...

In [61]:
# This bit takes the 'numCampaigns' most clicked campaigns and writes the results

sqlQuery = '''SELECT
A.campaign_id,
A.clientsname,
A.countryname,
A.devicename,
A.startdate,
A.enddate,
sum(A.numRecords) as totalImpressions,
B.numTotals
from adform.impressionsExtended as A
inner join (
  SELECT  campaign_id,
  sum(numRecords) AS numTotals,
  rank() over (order by numTotals desc) as rnk
  from adform.impressionsExtended
  group by 1
) as B
on A.campaign_id = B.campaign_id
where B.rnk <= {}
group by 1,2,3,4,5,6,B.numTotals
order by numTotals DESC'''.format(numCampaigns)


df        = rs.query2DF(sqlQuery)
groupedDF = df.groupby(df['campaign_id'])
idx       = 0;


# Get a list of the most clicked campaigns
vars2show = ['campaign_id', 'clientsname', 'numtotals', 'startdate', 'enddate']
df2 = df[vars2show].copy()
df2.drop_duplicates(inplace=True)
display(df2)

,campaign_id,clientsname,numtotals,startdate,enddate
0,897248,CoverGirl,104971036,2017-05-15 00:00:00.0,2017-06-30 23:59:00.0
358,886811,Sally Hansen,48491511,2017-04-25 00:00:00.0,2017-06-30 23:59:00.0
746,929928,Clairol Consumer,41799226,2017-06-15 00:00:00.0,2017-06-30 23:59:00.0
1480,886100,Sally Hansen,41189281,2017-04-17 00:00:00.0,2017-06-30 23:59:00.0
2177,886095,Sally Hansen,32643950,2017-05-01 00:00:00.0,2017-06-30 23:59:00.0
2856,888329,Sally Hansen,32176714,2017-05-08 00:00:00.0,2017-06-30 23:59:00.0
3269,894699,Rimmel,30504815,2017-05-10 00:00:00.0,2017-06-06 23:59:00.0
3833,912694,OPI Professional,28922186,2017-06-01 00:00:00.0,2017-06-30 23:59:00.0
4545,886846,Sally Hansen,20148263,2017-05-08 00:00:00.0,2017-06-30 23:59:00.0
5266,903883,Rimmel,19277041,2017-05-18 00:00:00.0,2017-06-10 23:59:00.0


In [62]:
for name, group in groupedDF:
    idx += 1;
    groupClientName = group['clientsname'].iloc[0]
    cu.printf('{} - Processing {} id {}...'.format(idx, groupClientName, name))
    print('Current campaign {} id {} got {} impressions'.format(groupClientName, \
        name, group['totalimpressions'].sum()))
    
    # Get the percentage for the break down
    totalCampaignClicks = group['totalimpressions'].sum();
    group['perct'] = 100.0*group['totalimpressions']/totalCampaignClicks;
    group.sort_values(['perct'], ascending=[0], inplace = True)
    
    # cutoff at 96%
    idx95p = group['perct'].cumsum() < cutOffPerc
    df95p  = group.loc[idx95p, :]

    fileName  = groupClientName + ' (id ' + str(name) + ').xlsx'
    

    display(df95p)

    # bokeh
    title  = groupClientName + \
        ' (id ' + str(name) + ') clicks: ' + str(totalCampaignClicks)
    labels = ['devicename','countryname']
    values = 'totalimpressions'
    hoverText ='totalimpressions'
    textFontSize = '10pt'

    donutChart = Donut(df95p, label=labels, values=values,
    text_font_size=textFontSize, hover_text=hoverText,
    title=title)
    output_notebook()
    show(donutChart)

Current campaign Philosophy id 875443 got 14802201 impressions


1 - Processing Philosophy id 875443...
/Users/carlos.aguilar/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/carlos.aguilar/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


,campaign_id,clientsname,countryname,devicename,startdate,enddate,totalimpressions,numtotals,perct
7859,875443,Philosophy,United States,Mobile,2017-04-06 00:00:00.0,2017-06-30 23:59:00.0,7137526,14802201,48.219356
7968,875443,Philosophy,United States,Desktop and Laptop,2017-04-06 00:00:00.0,2017-06-30 23:59:00.0,4150401,14802201,28.039080


/Users/carlos.aguilar/anaconda3/lib/python3.6/site-packages/bkcharts/builders/donut_builder.py:118: PerformanceWarning: indexing past lexsort depth may impact performance.
  chart = create_and_build(DonutBuilder, data, **kw)


Loading BokehJS ...

...

Loading BokehJS ...

Current campaign Sally Hansen id 941709 got 13191594 impressions


20 - Processing Sally Hansen id 941709...


,campaign_id,clientsname,countryname,devicename,startdate,enddate,totalimpressions,numtotals,perct
9296,941709,Sally Hansen,United States,Mobile,2017-07-05 00:00:00.0,2017-07-31 23:59:00.0,7022252,13191594,53.232778
9273,941709,Sally Hansen,United States,Desktop and Laptop,2017-07-05 00:00:00.0,2017-07-31 23:59:00.0,4616102,13191594,34.992754
9038,941709,Sally Hansen,United States,Tablet,2017-07-05 00:00:00.0,2017-07-31 23:59:00.0,587140,13191594,4.450865
9050,941709,Sally Hansen,United Kingdom,Desktop and Laptop,2017-07-05 00:00:00.0,2017-07-31 23:59:00.0,108721,13191594,0.824169
9040,941709,Sally Hansen,None,Desktop and Laptop,2017-07-05 00:00:00.0,2017-07-31 23:59:00.0,71935,13191594,0.545309
9060,941709,Sally Hansen,Australia,Desktop and Laptop,2017-07-05 00:00:00.0,2017-07-31 23:59:00.0,71579,13191594,0.542611
9042,941709,Sally Hansen,United States,Unknown,2017-07-05 00:00:00.0,2017-07-31 23:59:00.0,65791,13191594,0.498734
9275,941709,Sally Hansen,None,Mobile,2017-07-05 00:00:00.0,2017-07-31 23:59:00.0,60808,13191594,0.460960
8822,941709,Sally Hansen,Canada,Desktop and Laptop,2017-07-05 00:00:00.0,2017-07-31 23:59:00.0,41573,13191594,0.315148


Loading BokehJS ...

In [63]:


rs.close()